In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chart_studio.plotly as pl
import plotly.offline as of
import cufflinks as cf
import datetime as dt
%matplotlib inline

In [2]:
of.init_notebook_mode(connected = True)
cf.go_offline()

### Load Data

In [ ]:
donations = pd.read_csv('Donations.csv')

In [ ]:
donors = pd.read_csv('Donors.csv')

In [ ]:
projects = pd.read_csv('Projects.csv')

In [ ]:
resources = pd.read_csv('Resources.csv')

In [3]:
schools = pd.read_csv('Schools.csv')

In [6]:
teachers = pd.read_csv('Teachers.csv')

### See data dimentions

In [ ]:
print('Shape of donations dataframe is:' , donations.shape)
print('Shape of donors dataframe is:' , donors.shape)
print('Shape of projects dataframe is:' , projects.shape)
print('Shape of resources dataframe is:' , resources.shape)
print('Shape of schools dataframe is:' , schools.shape)
print('Shape of teachers dataframe is:' , teachers.shape)

### Create new data

In [ ]:
data = pd.merge(donations , projects , how='inner' , on = 'Project ID')
data2 = pd.merge(data , donors , how='inner' , on='Donor ID')
data3 = pd.merge(data2 , schools , how='inner' , on='School ID')
data4 = pd.merge(data3, teachers , how='inner' , on='Teacher ID')

In [ ]:
a = data4.columns.values.tolist()

In [4]:
s = schools['School State'].value_counts().sort_values(ascending = False).head(10)

In [5]:
s.iplot(kind='bar' , xTitle='States' , yTitle='Schools' , title='Schools with projects by state')

In [ ]:
s2 = data4.groupby('School State')['Donation Amount'].mean().sort_values(ascending=False).head(10)

In [ ]:
s2.iplot(kind='bar',
         xTitle='State',
         yTitle='AVG donation per project',
         title='States with major donations',
         colorscale='paired')

In [ ]:
mean = np.mean(data4['Donation Amount'].dropna())
median = np.median(data4['Donation Amount'].dropna())
percentiles = np.percentile(data4['Donation Amount'].dropna() ,[25,75])
minimum = data4['Donation Amount'].dropna().min()
maximum = data4['Donation Amount'].dropna().max()

print('mean donation amount is:' ,np.round(mean,2))
print('median donation amount is:' ,median)
print('25% and 75% donation amount is:' ,percentiles)
print('minimum donation amount is:' ,minimum)
print('maximum donation amount is:' ,maximum)

In [ ]:
x = np.sort(data4["Donation Amount"].dropna())
y = np.arange(1,len(x)+1)/len(x)
plt.plot(x,y,marker = '.')

In [ ]:
s3 = data4.groupby('Donor State')['Donation ID'].count().sort_values(ascending = False).head(15)

In [ ]:
s3.iplot(kind='bar' , xTitle='State',yTitle='Donations',title='Donations Count',colorscale='paired')

In [ ]:
s4 = schools['School State'].value_counts()
s5 = data4.groupby('Donor State')['Donation ID'].count()
df = pd.concat([s4,s5],axis=1,keys=['Projects','Donations'])

In [ ]:
df = df.dropna()

In [ ]:
df.iplot(kind='scatter',xTitle='Projects',
         yTitle='Donations',title='Projects vs Donations',
         symbol='x',colorscale='paired',mode='markers')

In [ ]:
slope,intercept = np.polyfit(df.Projects,df.Donations,1)
x = np.array([df.Projects.min(),df.Projects.max()])
y = slope*x + intercept
plt.plot(x,y)

In [ ]:
df.plot.scatter(x='Projects' , y='Donations')
slope,intercept = np.polyfit(df.Projects,df.Donations,1)
x = np.array([df.Projects.min(),df.Projects.max()])
y = slope*x + intercept
plt.plot(x,y)
plt.tight_layout()
plt.margins(0.05)

In [ ]:
s6 = data4["Project Type"].value_counts()
s7 = data4.groupby('Project Type')['Donation Amount'].sum().astype(int)

In [ ]:
plt.subplot(2,1,1)
plt.pie(s6 , startangle=90)
plt.subplot(2,1,2)
plt.pie(s7 , startangle=90)
plt.tight_layout()
plt.margins(0.05)
fig = plt.gcf()
fig.set_size_inches(25,15)

In [ ]:
data4['Project Subject Category Tree'].nunique()

In [ ]:
s8 = data4.groupby('Project Subject Category Tree')['Donation Amount'].sum().astype(int).sort_values(ascending = False).head(15)
s9 = s8/1000000
s9.iplot(kind="bar",
         xTitle='Project sub categories',
         yTitle='Donations amount (millions)',
         title='Donations by project subject',
         colorscale='paired')

In [ ]:
data4[['Project Posted Date' , 'Project Fully Funded Date']].isnull().sum()

In [ ]:
data4['Project Posted Date'] = pd.to_datetime(data4['Project Posted Date'])
data4['Project Fully Funded Date'] = pd.to_datetime(data4['Project Fully Funded Date'])
data4['Funding Time'] = data4['Project Fully Funded Date'] - data4['Project Posted Date'] 
data4[['Funding Time','Project Posted Date' , 'Project Fully Funded Date']].head()

In [ ]:
data4[['Funding Time','Project Posted Date' , 'Project Fully Funded Date']].isnull().sum()

In [ ]:
data5 = data4[pd.notnull(data4['Funding Time'])]
data5[['Funding Time','Project Posted Date' , 'Project Fully Funded Date']].isnull().sum()

In [ ]:
import datetime as dt
data5['Funding Time'] = data5['Funding Time'].dt.days

In [ ]:
wrong_overall_mean_time = data5['Funding Time'].mean()
wrong_overall_mean_time

In [ ]:
overall_mean_time = data5.groupby('Project ID')['Funding Time'].mean()
output = overall_mean_time.mean()

In [ ]:
#Average funding time for each state
state_project_funding_time = data5.groupby(['School State' , 'Project ID'])['Funding Time'].mean()
state_project_funding_time

In [ ]:
state_average_project_funding_time = state_project_funding_time.groupby('School State').mean()
state_average_project_funding_time.round(0)

In [ ]:
fast = state_average_project_funding_time.round(0)
fast[fast<32].sort_values().head(10)

In [ ]:
fast_funding = fast[fast<32].sort_values().head(10)
fast_funding.iplot(kind='bar',
                   xTitle='States',
                   yTitle='fully funding time(in days)',
                   title='states that fund projects earlier than others',
                   colorscale='paired')

In [ ]:
slow = state_average_project_funding_time.round(0)
slow[slow>32].sort_values(ascending = False).head(10)

In [ ]:
slow_funding = slow[slow>32].sort_values(ascending = False).head(10)
slow_funding.iplot(kind='bar',
                   xTitle='States',
                   yTitle='fully funding time(in days)',
                   title='states that fund projects earlier than others')